In [2]:
# Importer les bibliothèques nécessaires
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# 1. Créer la session Spark et se connecter à PostgreSQL
spark = SparkSession.builder \
    .appName("SentimentAnalysis") \
    .config("spark.jars", "/usr/local/spark/jars/postgresql-42.3.9.jar") \
    .getOrCreate()

# URL JDBC et propriétés de connexion
jdbc_url = "jdbc:postgresql://postgres:5432/DB_Mastodon"
properties = {
    "user": "fadi",
    "password": "fadi",
    "driver": "org.postgresql.Driver"
}

# Charger les données de PostgreSQL
df = spark.read.jdbc(url=jdbc_url, table="toots", properties=properties)



# 2. Nettoyage du contenu HTML
cleaned_data = df.withColumn("clean_content", regexp_replace(col("content"), "<[^>]+>", ""))
cleaned_data.select("clean_content").show(truncate=False)

# 3. Prétraitement du texte
# Tokenisation du texte
tokenizer = Tokenizer(inputCol="clean_content", outputCol="words")
tokenized_data = tokenizer.transform(cleaned_data)

# Suppression des mots inutiles (stopwords)
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
filtered_data = remover.transform(tokenized_data)

# Vectorisation avec CountVectorizer
vectorizer = CountVectorizer(inputCol="filtered_words", outputCol="raw_features")
vectorized_data = vectorizer.fit(filtered_data).transform(filtered_data)

# Appliquer TF-IDF
idf = IDF(inputCol="raw_features", outputCol="features")
rescaled_data = idf.fit(vectorized_data).transform(vectorized_data)

# 4. Entraîner le modèle de classification (Logistic Regression)
# Si vous avez une colonne "sentiment" dans vos données pour la classification
# Sinon, vous devrez ajouter des étiquettes de sentiments manuellement ou obtenir un dataset avec des labels
indexer = StringIndexer(inputCol="sentiment", outputCol="label")
labeled_data = indexer.fit(rescaled_data).transform(rescaled_data)

# Diviser les données en ensemble d'entraînement et de test
train_data, test_data = labeled_data.randomSplit([0.8, 0.2], seed=1234)

# Créer et entraîner un modèle de régression logistique
lr = LogisticRegression(featuresCol="features", labelCol="label")
model = lr.fit(train_data)

# Prédictions sur l'ensemble de test
predictions = model.transform(test_data)

# 5. Évaluer le modèle
# Créer un évaluateur pour calculer l'exactitude du modèle
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Précision du modèle : {accuracy}")

# Afficher les prédictions
predictions.select("clean_content", "label", "prediction").show(truncate=False)

# 6. Sauvegarder les résultats dans PostgreSQL
# Sélectionner les colonnes pertinentes à sauvegarder
results = predictions.select("id", "clean_content", "prediction")

# Sauvegarder les résultats dans une nouvelle table PostgreSQL
results.write.jdbc(url=jdbc_url, table="toot_sentiment_analysis", mode="append", properties=properties)

# Fermer la session Spark
spark.stop()


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|clean_content                                                                                                                                                                                                                                                                                                                                                                                                    |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Py4JJavaError: An error occurred while calling o314.fit.
: org.apache.spark.SparkException: Input column sentiment does not exist.
	at org.apache.spark.ml.feature.StringIndexerBase.$anonfun$validateAndTransformSchema$2(StringIndexer.scala:128)
	at scala.collection.TraversableLike.$anonfun$flatMap$1(TraversableLike.scala:293)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
	at scala.collection.TraversableLike.flatMap(TraversableLike.scala:293)
	at scala.collection.TraversableLike.flatMap$(TraversableLike.scala:290)
	at scala.collection.mutable.ArrayOps$ofRef.flatMap(ArrayOps.scala:198)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema(StringIndexer.scala:123)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema$(StringIndexer.scala:115)
	at org.apache.spark